In [1]:
from tensorflow import keras
from keras.models import Model
from keras import layers
from keras.layers import Dense, Input, BatchNormalization, Activation
from keras.layers import Conv2D, SeparableConv2D, MaxPooling2D, GlobalAveragePooling2D, GlobalMaxPooling2D
#from keras.applications.imagenet_utils import _obtain_input_shape

from sklearn.model_selection import KFold
from keras.datasets import mnist
from PIL import Image
# for creating a one hot vector for labels

from IPython.display import display, Image
#import the models
from keras import Model
#add layers
from keras import layers
#add optimizer
from keras import optimizers
#add loss function 
from keras import losses

import keras
import numpy as np
import pandas as pd 
import os
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.layers import Rescaling
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D

2025-11-08 04:01:11.937255: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762574472.153010      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762574472.217966      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [2]:
from keras.models import Model
from keras.layers import (
    Input, Conv2D, SeparableConv2D, MaxPooling2D,
    GlobalAveragePooling2D, GlobalMaxPooling2D, BatchNormalization,
    Activation, Dense, Dropout, Add, Rescaling, SpatialDropout2D, concatenate
)
from keras.optimizers import Adam
from keras.losses import CategoricalCrossentropy
from keras import regularizers, mixed_precision
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

# ✅ Enable mixed precision for faster GPU training (optional, works on Kaggle GPUs)
mixed_precision.set_global_policy('mixed_float16')

# ===== CONFIG =====
input_shape = (299, 299, 3)
classes = 7
weight_decay = 1e-4
drop_rate = 0.5
reg = regularizers.l2(weight_decay)

# ===== MODEL DEFINITION =====
def ImprovedXception():
    img_input = Input(shape=input_shape)
    x = Rescaling(1./255)(img_input)

    # ===== Entry Flow =====
    x = Conv2D(32, (3,3), strides=(2,2), use_bias=False, kernel_regularizer=reg)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(64, (3,3), use_bias=False, kernel_regularizer=reg)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Block 1
    residual = Conv2D(128, (1,1), strides=(2,2), padding='same', use_bias=False,   kernel_regularizer=reg)(x)
    residual = BatchNormalization()(residual)

    x = SeparableConv2D(128, (3,3), padding='same', use_bias=False,  depthwise_regularizer=reg, pointwise_regularizer=reg)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = SeparableConv2D(128, (3,3), padding='same', use_bias=False,  depthwise_regularizer=reg, pointwise_regularizer=reg)(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((3,3), strides=(2,2), padding='same')(x)
    x = Add()([x, residual])

    # Block 2
    residual = Conv2D(256, (1,1), strides=(2,2), padding='same', use_bias=False, kernel_regularizer=reg)(x)
    residual = BatchNormalization()(residual)

    x = Activation('relu')(x)
    x = SeparableConv2D(256, (3,3), padding='same', use_bias=False, depthwise_regularizer=reg, pointwise_regularizer=reg)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(256, (3,3), padding='same', use_bias=False, depthwise_regularizer=reg, pointwise_regularizer=reg)(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((3,3), strides=(2,2), padding='same')(x)
    x = Add()([x, residual])

    # Block 3
    residual = Conv2D(728, (1,1), strides=(2,2), padding='same', use_bias=False, kernel_regularizer=reg)(x)
    residual = BatchNormalization()(residual)

    x = Activation('relu')(x)
    x = SeparableConv2D(728, (3,3), padding='same', use_bias=False, depthwise_regularizer=reg, pointwise_regularizer=reg)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(728, (3,3), padding='same', use_bias=False, depthwise_regularizer=reg, pointwise_regularizer=reg)(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((3,3), strides=(2,2), padding='same')(x)
    x = Add()([x, residual])

    # ===== Middle Flow (8 times) =====
    for _ in range(8):
        residual = x
        x = Activation('relu')(x)
        x = SeparableConv2D(728, (3,3), padding='same', use_bias=False, depthwise_regularizer=reg, pointwise_regularizer=reg)(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = SeparableConv2D(728, (3,3), padding='same', use_bias=False, depthwise_regularizer=reg, pointwise_regularizer=reg)(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = SeparableConv2D(728, (3,3), padding='same', use_bias=False, depthwise_regularizer=reg, pointwise_regularizer=reg)(x)
        x = BatchNormalization()(x)
        x = SpatialDropout2D(0.2)(x)  # 🔥 Helps reduce overfitting
        x = Add()([x, residual])

    # ===== Exit Flow =====
    residual = Conv2D(1024, (1,1), strides=(2,2), padding='same', use_bias=False, kernel_regularizer=reg)(x)
    residual = BatchNormalization()(residual)

    x = Activation('relu')(x)
    x = SeparableConv2D(728, (3,3), padding='same', use_bias=False, depthwise_regularizer=reg, pointwise_regularizer=reg)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(1024, (3,3), padding='same', use_bias=False, depthwise_regularizer=reg, pointwise_regularizer=reg)(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((3,3), strides=(2,2), padding='same')(x)
    x = Add()([x, residual])

    x = SeparableConv2D(1536, (3,3), padding='same', use_bias=False, depthwise_regularizer=reg, pointwise_regularizer=reg)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = SeparableConv2D(2048, (3,3), padding='same', use_bias=False, depthwise_regularizer=reg, pointwise_regularizer=reg)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # ===== Classifier =====
    avg_pool = GlobalAveragePooling2D()(x)
    max_pool = GlobalMaxPooling2D()(x)
    x = concatenate([avg_pool, max_pool])
    x = Dropout(drop_rate)(x)
    x = Dense(classes, activation='softmax', dtype='float32')(x)

    model = Model(img_input, x, name='improved_xception')
    return model


# ===== Compile the Model =====
model = ImprovedXception()

loss_fn = CategoricalCrossentropy(label_smoothing=0.1)  # 🔥 Prevents overconfidence
optimizer = Adam(learning_rate=1e-4)

model.compile(
    optimizer=optimizer,
    loss=loss_fn,
    metrics=['accuracy']
)

model.summary()


# ===== Callbacks for Stable Training =====
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=15,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    verbose=1,
    min_lr=1e-6
)

# ===== Example Training Call =====
# model.fit(
#     train_generator,
#     validation_data=val_generator,
#     epochs=100,
#     callbacks=[early_stop, reduce_lr]
# )


I0000 00:00:1762574489.110756      48 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1762574489.111524      48 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Model: "improved_xception"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 299, 299,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cast (Cast)         │ (None, 299, 299,  │          0 │ input_layer[0][0] │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 299, 299,  │          0 │ cast[0][0]        │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 149, 149,  │        864 │ rescaling[0][0]   │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 149, 149,  │        128 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 149, 149,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 147, 147,  │     18,432 │ activation[0][0]  │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 147, 147,  │        256 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 147, 147,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ separable_conv2d    │ (None, 147, 147,  │      8,768 │ activation_1[0][… │
│ (SeparableConv2D)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 147, 147,  │        512 │ separable_conv2d… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 147, 147,  │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ separable_conv2d_1  │ (None, 147, 147,  │     17,536 │ activation_2[0][… │
│ (SeparableConv2D)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 147, 147,  │        512 │ separable_conv2d… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 74, 74,    │      8,192 │ activation_1[0][… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 74, 74,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 74, 74,    │        512 │ conv2d_2[0][0]  

 Total params: 20,890,159 (79.69 MB)

 Trainable params: 20,835,631 (79.48 MB)

 Non-trainable params: 54,528 (213.00 KB)

In [3]:
import os
x = []
label = []
x_validation = []
label_validation = []
x_test = []
label_test = []
img_size = 299


image_link = "/kaggle/input/tea-leaf-dataset-data-in-brief/teaLeafBD/teaLeafBD/teaLeafBD/1. Tea algal leaf spot"
image_list = os.listdir(image_link)
print(len(image_list))

for i in range(0, len(os.listdir(image_link))):
    if i < 334:
        image = cv2.imread(image_link + '/' + image_list[i])
        image = cv2.resize(image , ( img_size , img_size ))
        x.append(image)
        label.append([0])
    elif i < 375:
        image = cv2.imread(image_link + '/' + image_list[i])
        image = cv2.resize(image , ( img_size , img_size ))
        x_validation.append(image)
        label_validation.append([0])
    else:
        image = cv2.imread(image_link + '/' + image_list[i])
        image = cv2.resize(image , ( img_size , img_size ))
        x_test.append(image)
        label_test.append([0])
       
        

print(len(x))
print(len(x_validation))
print(len(x_test))

418
334
41
43


In [4]:
"""
x = []
label = []
x_validation = []
label_validation = []
x_test = []
label_test = []
img_size = 299
"""

image_link = "/kaggle/input/tea-leaf-dataset-data-in-brief/teaLeafBD/teaLeafBD/teaLeafBD/2. Brown Blight"
image_list = os.listdir(image_link)
print(len(image_list))

for i in range(0, len(os.listdir(image_link))):
    if i < 405:
        image = cv2.imread(image_link + '/' + image_list[i])
        image = cv2.resize(image , ( img_size , img_size ))
        x.append(image)
        label.append([1])
    elif i < 455:
        image = cv2.imread(image_link + '/' + image_list[i])
        image = cv2.resize(image , ( img_size , img_size ))
        x_validation.append(image)
        label_validation.append([1])
    else:
        image = cv2.imread(image_link + '/' + image_list[i])
        image = cv2.resize(image , ( img_size , img_size ))
        x_test.append(image)
        label_test.append([1])
       
        

print(len(x))
print(len(x_validation))
print(len(x_test))

506
739
91
94


In [5]:
"""
x = []
label = []
x_validation = []
label_validation = []
x_test = []
label_test = []
img_size = 299
"""

image_link = "/kaggle/input/tea-leaf-dataset-data-in-brief/teaLeafBD/teaLeafBD/teaLeafBD/3. Gray Blight"
image_list = os.listdir(image_link)
print(len(image_list))

for i in range(0, len(os.listdir(image_link))):
    if i < 810:
        image = cv2.imread(image_link + '/' + image_list[i])
        image = cv2.resize(image , ( img_size , img_size ))
        x.append(image)
        label.append([2])
    elif i < 911:
        image = cv2.imread(image_link + '/' + image_list[i])
        image = cv2.resize(image , ( img_size , img_size ))
        x_validation.append(image)
        label_validation.append([2])
    else:
        image = cv2.imread(image_link + '/' + image_list[i])
        image = cv2.resize(image , ( img_size , img_size ))
        x_test.append(image)
        label_test.append([2])
       
        

print(len(x))
print(len(x_validation))
print(len(x_test))

1013
1549
192
196


In [6]:
"""
x = []
label = []
x_validation = []
label_validation = []
x_test = []
label_test = []
img_size = 299
"""

image_link = "/kaggle/input/tea-leaf-dataset-data-in-brief/teaLeafBD/teaLeafBD/teaLeafBD/4. Helopeltis"
image_list = os.listdir(image_link)
print(len(image_list))

for i in range(0, len(os.listdir(image_link))):
    if i < 486:
        image = cv2.imread(image_link + '/' + image_list[i])
        image = cv2.resize(image , ( img_size , img_size ))
        x.append(image)
        label.append([3])
    elif i < 546:
        image = cv2.imread(image_link + '/' + image_list[i])
        image = cv2.resize(image , ( img_size , img_size ))
        x_validation.append(image)
        label_validation.append([3])
    else:
        image = cv2.imread(image_link + '/' + image_list[i])
        image = cv2.resize(image , ( img_size , img_size ))
        x_test.append(image)
        label_test.append([3])
       
        

print(len(x))
print(len(x_validation))
print(len(x_test))

607
2035
252
257


In [7]:
"""
x = []
label = []
x_validation = []
label_validation = []
x_test = []
label_test = []
img_size = 299
"""

image_link = "/kaggle/input/tea-leaf-dataset-data-in-brief/teaLeafBD/teaLeafBD/teaLeafBD/5. Red spider"
image_list = os.listdir(image_link)
print(len(image_list))

for i in range(0, len(os.listdir(image_link))):
    if i < 412:
        image = cv2.imread(image_link + '/' + image_list[i])
        image = cv2.resize(image , ( img_size , img_size ))
        x.append(image)
        label.append([4])
    elif i < 464:
        image = cv2.imread(image_link + '/' + image_list[i])
        image = cv2.resize(image , ( img_size , img_size ))
        x_validation.append(image)
        label_validation.append([4])
    else:
        image = cv2.imread(image_link + '/' + image_list[i])
        image = cv2.resize(image , ( img_size , img_size ))
        x_test.append(image)
        label_test.append([4])
       
        

print(len(x))
print(len(x_validation))
print(len(x_test))

515
2447
304
308


In [8]:
"""
x = []
label = []
x_validation = []
label_validation = []
x_test = []
label_test = []
img_size = 299
"""

image_link = "/kaggle/input/tea-leaf-dataset-data-in-brief/teaLeafBD/teaLeafBD/teaLeafBD/6. Green mirid bug"
image_list = os.listdir(image_link)
print(len(image_list))

for i in range(0, len(os.listdir(image_link))):
    if i < 1025:
        image = cv2.imread(image_link + '/' + image_list[i])
        image = cv2.resize(image , ( img_size , img_size ))
        x.append(image)
        label.append([5])
    elif i < 1153:
        image = cv2.imread(image_link + '/' + image_list[i])
        image = cv2.resize(image , ( img_size , img_size ))
        x_validation.append(image)
        label_validation.append([5])
    else:
        image = cv2.imread(image_link + '/' + image_list[i])
        image = cv2.resize(image , ( img_size , img_size ))
        x_test.append(image)
        label_test.append([5])
       
        

print(len(x))
print(len(x_validation))
print(len(x_test))

1282
3472
432
437


In [9]:
"""
x = []
label = []
x_validation = []
label_validation = []
x_test = []
label_test = []
img_size = 299
"""

image_link = "/kaggle/input/tea-leaf-dataset-data-in-brief/teaLeafBD/teaLeafBD/teaLeafBD/7. Healthy leaf"
image_list = os.listdir(image_link)
print(len(image_list))

for i in range(0, len(os.listdir(image_link))):
    if i < 748:
        image = cv2.imread(image_link + '/' + image_list[i])
        image = cv2.resize(image , ( img_size , img_size ))
        x.append(image)
        label.append([6])
    elif i < 842:
        image = cv2.imread(image_link + '/' + image_list[i])
        image = cv2.resize(image , ( img_size , img_size ))
        x_validation.append(image)
        label_validation.append([6])
    else:
        image = cv2.imread(image_link + '/' + image_list[i])
        image = cv2.resize(image , ( img_size , img_size ))
        x_test.append(image)
        label_test.append([6])
       
        

print(len(x))
print(len(x_validation))
print(len(x_test))

935
4220
526
530


In [10]:
x = np.asarray(x)
x_validation = np.asarray(x_validation)
x_test = np.asarray(x_test)

print(len(x)," ", len(x_validation)," ", len(x_test))

4220   526   530


In [11]:
from tensorflow import keras


label = keras.utils.to_categorical(label, classes)
print(len(label))
print(label)


label_validation = keras.utils.to_categorical(label_validation, classes)
print(len(label_validation))
print(label_validation)
print(label_validation[-1])


label_test = keras.utils.to_categorical(label_test, classes )
print(len(label_test))
print(label_test)
print(label_test[-1])



4220
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
526
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
[0. 0. 0. 0. 0. 0. 1.]
530
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
[0. 0. 0. 0. 0. 0. 1.]


In [12]:
#early_stop  = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience= 90 , verbose=0, mode='max')
check_point = tf.keras.callbacks.ModelCheckpoint('./tea_leaf_images_dataset.keras', monitor='val_accuracy', save_best_only=True, mode='max')
history = model.fit(x , label , validation_data = (x_validation , label_validation ), epochs = 100 , callbacks=[early_stop, check_point, reduce_lr])
    

print("skip")

Epoch 1/100


I0000 00:00:1762574632.153165     111 service.cc:148] XLA service 0x7e6b30032580 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1762574632.154023     111 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1762574632.154053     111 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1762574636.507541     111 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-11-08 04:04:11.667862: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng0{} for conv (f16[32,149,149,32]{3,2,1,0}, u8[0]{0}) custom-call(f16[32,147,147,64]{3,2,1,0}, f16[64,3,3,32]{3,2,1,0}), window={size=3x3}, dim_labels=b01f_o01i->b01f, custom_call_target="__cudnn$convBackwardInput", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","

131/132 ━━━━━━━━━━━━━━━━━━━━ 0s 213ms/step - accuracy: 0.2645 - loss: 4.7862

2025-11-08 04:05:35.505654: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng0{} for conv (f16[28,149,149,32]{3,2,1,0}, u8[0]{0}) custom-call(f16[28,147,147,64]{3,2,1,0}, f16[64,3,3,32]{3,2,1,0}), window={size=3x3}, dim_labels=b01f_o01i->b01f, custom_call_target="__cudnn$convBackwardInput", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]} is taking a while...
2025-11-08 04:05:35.822030: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.316556776s
Trying algorithm eng0{} for conv (f16[28,149,149,32]{3,2,1,0}, u8[0]{0}) custom-call(f16[28,147,147,64]{3,2,1,0}, f16[64,3,3,32]{3,2,1,0}), window={size=3x3}, dim_labels=b01f_o01i->b01f, custom_call_target="__cudnn$convBackwardInput", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","con

132/132 ━━━━━━━━━━━━━━━━━━━━ 184s 750ms/step - accuracy: 0.2656 - loss: 4.7827 - val_accuracy: 0.1787 - val_loss: 4.4792 - learning_rate: 1.0000e-04
Epoch 2/100
132/132 ━━━━━━━━━━━━━━━━━━━━ 32s 240ms/step - accuracy: 0.4414 - loss: 4.2595 - val_accuracy: 0.1787 - val_loss: 4.4746 - learning_rate: 1.0000e-04
Epoch 3/100
132/132 ━━━━━━━━━━━━━━━━━━━━ 31s 232ms/step - accuracy: 0.4884 - loss: 4.0831 - val_accuracy: 0.1787 - val_loss: 4.4704 - learning_rate: 1.0000e-04
Epoch 4/100
132/132 ━━━━━━━━━━━━━━━━━━━━ 30s 229ms/step - accuracy: 0.5519 - loss: 3.9151 - val_accuracy: 0.1787 - val_loss: 4.4746 - learning_rate: 1.0000e-04
Epoch 5/100
132/132 ━━━━━━━━━━━━━━━━━━━━ 33s 252ms/step - accuracy: 0.6056 - loss: 3.7982 - val_accuracy: 0.2281 - val_loss: 4.4864 - learning_rate: 1.0000e-04
Epoch 6/100
132/132 ━━━━━━━━━━━━━━━━━━━━ 33s 250ms/step - accuracy: 0.6707 - loss: 3.6767 - val_accuracy: 0.5247 - val_loss: 3.9324 - learning_rate: 1.0000e-04
Epoch 7/100
132/132 ━━━━━━━━━━━━━━━━━━━━ 33s 251ms/

In [13]:
extract_saved_Xception_model = tf.keras.models.load_model('/kaggle/working/tea_leaf_images_dataset.keras') 
extract_saved_Xception_model.summary()

Model: "improved_xception"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 299, 299,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cast_2 (Cast)       │ (None, 299, 299,  │          0 │ input_layer[0][0] │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 299, 299,  │          0 │ cast_2[0][0]      │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 149, 149,  │        864 │ rescaling[0][0]   │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 149, 149,  │        128 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 149, 149,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 147, 147,  │     18,432 │ activation[0][0]  │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 147, 147,  │        256 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 147, 147,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ separable_conv2d    │ (None, 147, 147,  │      8,768 │ activation_1[0][… │
│ (SeparableConv2D)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 147, 147,  │        512 │ separable_conv2d… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 147, 147,  │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ separable_conv2d_1  │ (None, 147, 147,  │     17,536 │ activation_2[0][… │
│ (SeparableConv2D)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 147, 147,  │        512 │ separable_conv2d… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 74, 74,    │      8,192 │ activation_1[0][… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 74, 74,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 74, 74,    │        512 │ conv2d_2[0][0]  

 Total params: 62,561,427 (238.65 MB)

 Trainable params: 20,835,631 (79.48 MB)

 Non-trainable params: 54,528 (213.00 KB)

 Optimizer params: 41,671,268 (158.96 MB)

In [14]:
loss, acc = extract_saved_Xception_model.evaluate(x_test, label_test, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

17/17 - 15s - 867ms/step - accuracy: 0.8472 - loss: 1.6531
Restored model, accuracy: 84.72%


In [ ]:
while(True):
    a = 5